# Use Case: Interactive Chat Experiance with Documents

We have a asked to create a chat experiance that allows our users to ask questions of our documents to enhance productivity and efficiency, providing a seamless and intuitive way to interact with and extract information from digital files.

Specifically for the [State of Michigan EMS Protocol Suite](https://www.michigan.gov/mdhhs/inside-mdhhs/legislationpolicy/ems/protocols/michigan-protocols) which contains:
- **10** documents (PDF)
- Containing a total of **405 pages** of text

### The Snowflake Features we use to build the solution are:
- [Snowflake Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks)
- [Snowpark for Python](https://docs.snowflake.com/en/developer-guide/snowpark/python/index)
- [Directory Tables](https://docs.snowflake.com/en/user-guide/data-load-dirtables)
- [Cortex Search](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview)
- [Cortex LLM Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)
- [Streamlit in Snowflake](https://www.snowflake.com/en/data-cloud/overview/streamlit-in-snowflake/)

# Prerequisites :wrench:
## :blue[Step 1: Add Required Python Packages]:snake:
#### :red[*Skip this step if you imported this notebook from Git. The `environment.yml` file handles package installation.]
Add the following Python packages by clicking `Packages` button in the upper right of the notebook, and searching for them by name.

```
snowflake
```
```
snowflake-ml-python
```

***

>> #### :bulb: Did you know?
>> Snowflake features a native integration with [Anaconda](https://www.anaconda.com/) for package and dependency management out of the box. This includes a curated [Anaconda Channel](https://repo.anaconda.com/pkgs/snowflake/) specifically for Snowflake with thousands of the most popular Python packages. There is no additional cost for such use of the Anaconda packages.
---

## :blue[Step 2: Load PDF Documents to the Notebook] :page_with_curl:

## How to add files to a Notebook from local computer
1. In the **Files** tab, to the left of your notebook, we can use the `+` icon to select files to upload.
1. **Before** adding the documeents:
    - Locate the field at the bottom of the upload interface named __'Specify the path to an existing folder or create a new one'__ 
    - Type in `MI_EMS_PROTO`
    - This is the folder name we will load our PDFs into.
1. Click **Browse** and select the PDF files or drag-and-drop them into the dialog.
1. Finally, click **Upload** to upload your files.

***
>> #### :bulb: Did you know?
>> - Every [Notebook](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks) in Snowflake has an internal filesystem for working with files that is only avaiable to the notebook. 
>>      - The files are stored in an internal stage which represents your notebook environment. 
>>      - Files stored in this internal stage persist between sessions.
>> - Your files can be viewed in the `Files` pane on the **left side of the notebook**.
---


# Setup Snowflake Eviroment :female-technologist:
## :blue[Step 1: Start the Notebook Session] :large_green_circle:
With the prerequisites completed, we are ready to start building the solution. We will start the notebook and import the required packages.

1. Click the `Start` button in the upper right of this notebook to start the notebook session.
1. Import packages and functions that we need for our solution.

In [ ]:
# import packages
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session
from snowflake.cortex import complete, extract_answer, summarize, translate

# get Snowflake session
session = get_active_session()
# The entry point of the Snowflake Core Python APIs that manage the Snowflake objects.
root = Root(session)

# Show the current session conetext
# By default the session context will have the database and schema where the Notebook is saved 
# and the warehouse your notebook is running on.
print("Current Role: ", session.get_current_role())
print("Current Warehouse: ", session.get_current_warehouse())
print("Current Database: ", session.get_current_database())
print("Current Schema: ", session.get_current_schema())
print("Current Fully Qualified Schema: ", session.get_fully_qualified_current_schema())

## :blue[Step 2: Create resources (or use an *existing* resources) for this solution] :open_file_folder: :gear:
The objects we will be creating require a schema to be stored in. We also need a virtual warehouse for the solution to use.

### In the next cells we will...
1. Assign the names of the following resources to Python variables
    * Database
    * Schema
    * Virtual Warehouse
1. Use **SQL** to update the session context by referencing our **Python** variables.
    * If a resource name/type doesn't exist, the SQL code will attempt to create them.

In [ ]:
# define variables in Python
# variables will be accesseable thoughout the rest of the notebook - even in SQL cells! 
solution_database = 'demo_ems_rag_app'
solution_schema = 'app'
query_warehouse = 'demo_ems_rag_app_wh'

# helper variable
fully_qualified_solution_schema = f"{solution_database}.{solution_schema}"

# display Python variable definitions
print("Variable Values:")
print(f"""solution_database = {solution_database}""")
print(f"""solution_schema = {solution_schema}""")
print(f"""query_warehouse = {query_warehouse}""")
print(f"""fully_qualified_solution_schema = {fully_qualified_solution_schema}""")

In [ ]:
-- create enviroment resources if they do not already exist
CREATE DATABASE IF NOT EXISTS IDENTIFIER('{{solution_database}}');
CREATE SCHEMA IF NOT EXISTS IDENTIFIER('{{fully_qualified_solution_schema}}');

CREATE WAREHOUSE IF NOT EXISTS IDENTIFIER('{{query_warehouse}}') WITH
     WAREHOUSE_SIZE='X-SMALL'
     AUTO_SUSPEND = 120
     AUTO_RESUME = TRUE
     INITIALLY_SUSPENDED=TRUE;

-- set session context
USE WAREHOUSE IDENTIFIER('{{query_warehouse}}');
USE SCHEMA IDENTIFIER('{{fully_qualified_solution_schema}}');

>> #### :bulb: Did you know?
>> - The `CREATE DATABASE` statement creates a database. 
>>      - All Snowflake databases automatically includes a schema named `PUBLIC`.
>> - The `CREATE WAREHOUSE` statement creates an initially suspended warehouse.
>> - When a new database, schema, or virtual warehouse is created, the session context will implicitly update to the newly created resource.
>>      - This implicit behavior is unique to the `CREATE` statements.
>> - You can explicitly update the session context with `USE` statements. 

## :blue[Step 3: Validate the Session Context has been updated] :white_check_mark:
- When we run the cell below, we should see the session context has been updated to the database, schema, and warehouse above.
- Last time we used Python to check this. This time we will use SQL to check the session context.

In [ ]:
WITH session_details as (
SELECT 'Warehouse' as current_item, current_warehouse() as actual, upper('{{query_warehouse}}') as expected UNION
SELECT 'Database', current_database(), upper('{{solution_database}}') UNION
SELECT 'Schema', current_schema(), upper('{{solution_schema}}') UNION
SELECT 'Fully Qualified Schema', current_database() || '.' || current_schema(), upper('{{fully_qualified_solution_schema}}')
)
SELECT *, iff(actual = expected, '✅ - Pass', '❌ - Fail - Run the 3 cells above and try again.') as validated FROM session_details

## :blue[Step 4: Make the documents available outside of this notebook] :file_cabinet:
In order to build the solution, we need to make our documents accessible outside of this notebook. 
To do this, we will:
1. Create a permanent named stage in Snowflake to store the files.
1. Copy the files from the Notebook filesystem to that stage. 


In [ ]:
-- variable for name of Snowflake stage
SET v_stage_name = '{{fully_qualified_solution_schema}}.'||'docs';

-- create stage if not exists
CREATE STAGE IF NOT EXISTS IDENTIFIER($v_stage_name)
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

### Save the Notebook filesystem path to the document folder to a variable
- Here we define a Python function to search though the Notebook filesystem to find the path to document folder.
- Then we set that path to a variable.

In [ ]:
import os
import streamlit as st

# Define python function to search though the file system for the path to a matching directory
def list_matching_directories_recursively(path, match_name):
    """
    Recursively list all directories that match a specified directory name.
    
    Args:
    path (str): The root directory to start the search from.
    match_name (str): The name of the directory to match.
    
    Returns:
    list: A list of matching directory paths.
    """
    matching_directories = []  # Initialize an empty list to store matching directories
    
    # Walk through the directory tree
    for root, dirs, files in os.walk(path):
        # Iterate over the directories in the current root
        for dir in dirs:
            # Check if the directory name matches the specified name
            if dir == match_name:
                # If it matches, add the full path of the directory to the list
                matching_directories.append(os.path.join(root, dir))
    
    return matching_directories

In [ ]:
# Specify the path you want to start from and the directory name to match
document_dir = 'MI_EMS_PROTO'
path = os.getcwd()
match_name = document_dir

# Call the function to get a list of matching directories
matching_directories = list_matching_directories_recursively(path, match_name)

# return the path to the first matching directory (assumption is there is only one)
document_path = matching_directories[0]
st.info(f"""The Notebook filesystem path to the documents folder is:

**`{document_path}`**""")
st.info(f"Files available in the **`{document_dir}`** folder:")
os.listdir(document_path)

### Now we will use Python to save the documents to the named stage and print the results
- Note: This could be done via the Snowsight GUI, SnowSQL, SnowCLI, and a number of other methods as well

In [ ]:
import pandas as pd

# Define the stage location 
stage = "@DOCS"

# Define the files you want to upload to a stage
file = "*" # '*' is a wildcard for all files in the directory 

# Define the path
stage_location = "/"+os.path.split(document_path)[-1]

# Push the files to the stage with a PUT operation
put_result = session.file.put(f"{document_path}/{file}", 
                              f"{stage}{stage_location}", 
                                auto_compress=False, 
                                overwrite=True)

# Convert the result to a pandas DataFrame for readability
df = pd.DataFrame(put_result)

# Print the results array
st.write(df)

### Validate our files have been saved the stage by querying the Directory Table

In [ ]:
-- Register new files in the stage with the directory table
ALTER STAGE IDENTIFIER($v_stage_name) REFRESH;

-- add the @ character required by the directory table
SET formatted_stage_name = '@'||$v_stage_name;

-- query the directory table
SELECT * FROM DIRECTORY($formatted_stage_name);

# Build the Solution!
## :blue[Step 1: Process our unstructured documents for Cortex Search and the Chat Application] :hammer:

In order to make our documents searchable and usable by an LLM we need to:
1. Extract the text from our PDFs.
1. Split the text of the document up into smaller blocks of text. This is proccess is called **'chunking'**.
    - Note: There are [several strategies](https://medium.com/@anuragmishra_27746/five-levels-of-chunking-strategies-in-rag-notes-from-gregs-video-7b735895694d) for chunking text for RAG, we will be using [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/) from [LangChain](https://www.langchain.com/), a popular python libary for this task.


### In order to accomplish the above, we will next register a Python function in Snowflake. 
- This can make the function availble for other Snowflake users 
- The function can be called like any other function n SQL or Python

We will do this in SQL, however you can also register a Python function with Snowpark as well.

---
>> #### :thinking_face: Why bother breaking up the text in the first place?
>> Chunking text for Retrieval-Augmented Generation (RAG) improves:
>> 
>> - **Efficiency**: Faster indexing and retrieval.
>> - **Accuracy**: Better query-to-text matching.
>> - **Context Management**: Focused and relevant segments.
>> - **Memory Management**: Handles large texts within memory/token limits.
>> - **Parallel Processing**: Speeds up processing.
>> - **Reduced Redundancy**: Minimizes repetitive information.
---

To make the function, we will use the following popular Python packages:
- [langchain](https://pypi.org/project/langchain/) :parrot: :link:
- [PyPDF2](https://pypi.org/project/PyPDF2/) :page_facing_up:
- [snowflake-snowpark-python](https://pypi.org/project/snowflake-snowpark-python/) :snowflake:




In [ ]:
CREATE OR REPLACE FUNCTION pdf_text_chunker(file_url STRING, chunk_size INT, chunk_overlap INT)
RETURNS TABLE (chunk VARCHAR, page_number INT)
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
HANDLER = 'pdf_text_chunker'
PACKAGES = ('snowflake-snowpark-python', 'PyPDF2', 'langchain')
AS
$$
"""
This function reads a PDF file from a specified URL, extracts the text from each page,
and splits the text into chunks of a given size with a specified overlap. The function 
returns a table containing the text chunks and their corresponding page numbers.

Parameters:
    file_url (STRING): The URL of the PDF file to be processed.
    chunk_size (INT): The size of each text chunk.
    chunk_overlap (INT): The number of characters that overlap between consecutive chunks.

Returns:
    TABLE: A table with two columns:
        - chunk (VARCHAR): A chunk of text extracted from the PDF.
        - page_number (INT): The page number of the PDF from which the chunk was extracted.
"""
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from snowflake.snowpark.files import SnowflakeFile
import PyPDF2, io
import logging
import pandas as pd

# Define the main class for PDF text chunking
class pdf_text_chunker:

    # Function to read and extract text from a PDF file
    def read_pdf(self, file_url: str):
        logger = logging.getLogger("udf_logger")
        logger.info(f"Opening file {file_url}")

        # Open the PDF file from the provided URL
        with SnowflakeFile.open(file_url, 'rb') as f:
            buffer = io.BytesIO(f.readall())

        # Initialize the PDF reader
        reader = PyPDF2.PdfReader(buffer)
        texts_and_pages = []

        # Extract text from each page of the PDF
        for page_number, page in enumerate(reader.pages, start=1):
            try:
                text = page.extract_text().replace('\n', ' ') #.replace('\0', ' ')
                texts_and_pages.append((text, page_number))
            except:
                text = "Unable to Extract"
                logger.warn(f"Unable to extract from file {file_url}, page {page_number}")
                texts_and_pages.append((text, page_number))

        return texts_and_pages

    # Function to process the PDF and split text into chunks
    def process(self, file_url: str, chunk_size: int, chunk_overlap: int):
        # Read text and page numbers from the PDF file
        texts_and_pages = self.read_pdf(file_url)

        # Initialize the text splitter with given chunk size and overlap
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = chunk_size,  # Set the chunk size
            chunk_overlap = chunk_overlap,  # Set the chunk overlap
            length_function = len
        )

        chunks_with_pages = []
        
        # Split the text into chunks and associate with page numbers
        for text, page_number in texts_and_pages:
            chunks = text_splitter.split_text(text)
            for chunk in chunks:
                chunks_with_pages.append((chunk, page_number))
        
        # Create a DataFrame from the chunks and page numbers
        df = pd.DataFrame(chunks_with_pages, columns=['chunk', 'page_number'])

        # Yield chunks as tuples
        yield from df.itertuples(index=False, name=None)
$$;

### Next, we create a table that maps the documents to the text chunks
- This table will also have other useful metadata such as the page number the text chunk came from

In [ ]:
CREATE OR REPLACE TABLE docs_chunks_table 
CHANGE_TRACKING = TRUE
AS
    SELECT
        relative_path,
        '@DOCS/' || relative_path as relative_path_with_stage,
        split_part(relative_path, '/', -1) as file_name,
        replace(relative_path, file_name, '') as folder_path,
        -- preserve file title and page number by concatenating with the chunk
        CONCAT(object_construct('document_title', file_name, 'page', page_number)::string, ' : ', func.chunk) AS chunk,
        func.page_number
    FROM
        directory(@docs),
        TABLE(pdf_text_chunker(
                    file_url => build_scoped_file_url(@docs, relative_path),
                    chunk_size => 2000,
                    chunk_overlap => 300)
              ) AS func
    WHERE true
        AND folder_path = 'MI_EMS_PROTO/';

## Preview and explore the newly created table

In [ ]:
select 
    file_name,
    page_number,
    chunk,
    --snowflake.cortex.count_tokens('snowflake-arctic-embed-m', chunk) token_count,
    relative_path,
    folder_path,
    relative_path_with_stage
from docs_chunks_table 
order by file_name, page_number
limit 10

## Compare the number of pages per document to the number of chunks

In [ ]:
select 
    file_name,
    max(page_number) as pages,
    count(*) as chunk_count,
    chunk_count - pages as chunk_vs_page_count_difference
from docs_chunks_table
group by all
order by pages desc;

## :blue[Step 2: Create the Cortex Search Service] :mag:

[Cortex Search](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview) enables low-latency, high-quality “fuzzy” search over your Snowflake data. Cortex Search powers a broad array of search experiences for Snowflake users including Retrieval Augmented Generation (RAG) applications leveraging Large Language Models (LLMs).

Cortex Search gets you up and running with a hybrid (vector and keyword) search engine on your text data in minutes, without having to worry about:
- Embedding
- Infrastructure maintenance
- Search quality parameter tuning
- Ongoing index refreshes. 

This means you can spend less time on infrastructure and search quality tuning, and more time developing high-quality chat and search experiences using your data.


In [ ]:
DROP CORTEX SEARCH SERVICE IF EXISTS EMS_PROTO;
CREATE OR REPLACE CORTEX SEARCH SERVICE EMS_PROTO
    ON chunk
    ATTRIBUTES file_name, page_number
    WAREHOUSE = {{query_warehouse}}
    TARGET_LAG = '24 hour'
    AS (
    SELECT
        chunk,
        file_name,
        page_number,
        relative_path,
        relative_path_with_stage
    FROM docs_chunks_table 
    );

In [ ]:
DESCRIBE CORTEX SEARCH SERVICE EMS_PROTO;

## :blue[Step 3: Query the Cortex Search Service] :magic_wand:

Cortex Search offers both a Python and REST API, as well as the ability to preview results with SQL. We will be using the Python interface.

Let's explore what can be done with the the search results...

In [ ]:
import streamlit as st
import pandas as pd

user_search = "What phone number should I call to report suspected adult neglect?"

# define the search service
mi_ems_proto_search_service = (root
  .databases[solution_database]
  .schemas[solution_schema]
  .cortex_search_services["EMS_PROTO"]
)

# first query
# resp = mi_ems_proto_search_service.search(
#   query=user_search,
#   columns=["chunk", "file_name", "page_number","relative_path","relative_path_with_stage"],
#   #filter={"@eq": {"region": "North America"} },
#   limit=1
# )

resp = mi_ems_proto_search_service.search(
  query=user_search,
  columns=[],
  #filter={"@eq": {"region": "North America"} },
  limit=1
)

# second query
resp2 = mi_ems_proto_search_service.search(
  query=user_search,
  columns=["file_name", "page_number","relative_path_with_stage"],
  #filter={"@eq": {"region": "North America"} },
  limit=5
)

# display results
st.header("""'Chunk' of text from the documents that most closely matches the question:""")
st.markdown(f"""### :blue[_**'{user_search}'**_]""")
search_response = resp.to_json()
st.json(search_response)

st.header("Enterprise Document Search")
st.subheader("Alternatively, we can use the seach service to simply return a listing of files that most closely match search terms.")
search_response2 = pd.DataFrame(resp2.to_dict()['results'])
st.write(search_response2)

## Example: Passing Cortex Search results though addational Cortex LLM Functions

In [ ]:
# get responses
extracted_answer = extract_answer(search_response, user_search)
summarized_answer = summarize(search_response)
summary_in_spanish = translate(summarized_answer, 'en','es')

# display results
st.header("Built-in Snowflake Cortex LLM Function Examples")
st.markdown("---")

st.markdown("### [SNOWFLAKE.CORTEX.EXTRACT_ANSWER()](https://docs.snowflake.com/en/sql-reference/functions/extract_answer-snowflake-cortex)")
st.write(f"""**Result for the question: :green['{user_search}']**""")
st.json(extracted_answer)
st.markdown("---")

st.markdown("### [SNOWFLAKE.CORTEX.SUMMARIZE()](https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex)")
st.write(summarized_answer)
st.markdown("---")

st.markdown("### [SNOWFLAKE.CORTEX.TRANSLATE()](https://docs.snowflake.com/en/sql-reference/functions/translate-snowflake-cortex)")
st.write("**(Translation of the above summary into Spanish)**")
st.write(summary_in_spanish)

## [SNOWFLAKE.CORTEX.COMPLETE()](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)
Given a prompt, generates a response (completion) using your choice of supported language model.

`SNOWFLAKE.CORTEX.COMPLETE()` is the primary Cortex function that will enable our users to chat with our documents. 

With this function you can:
- Choose which Snowflake-hosted Large Language Model (LLM) to use.
- Provide the model instructions and context for the type of response you want.

**SQL Syntax**
```sql
SELECT SNOWFLAKE.CORTEX.COMPLETE(<model>, <prompt_or_history> [ , <options> ] ) AS RESPONSE;
```



In [ ]:
model = 'snowflake-llama-3.3-70b'
user_question = 'What phone number should I call to report suspected adult neglect?'
prompt_context = search_response

prompt = f"""
        [INST]
        You are a helpful AI chat assistant with RAG capabilities. When a user asks you a question,
        you will also be given context provided between <context> and </context> tags. 
        Ensure the answer is coherent, concise, and directly relevant to the user's question.

        If the user asks a generic question which cannot be answered with the given context or chat_history,
        just say "I don't know the answer to that question.
        
        Don't say phases like "according to the provided information".
        
        <context>          
        {prompt_context}
        </context>
        <question>  
        {user_question}
        </question>
        [/INST]
        Answer:
       """
cortex_reponse = complete(model, prompt)

st.header('Example: Simulated Chat Experience with CORTEX.COMPLETE()')

st.markdown("---")

with st.chat_message("ai"):
    st.write("👋 Hi, I am an AI assistent that can answer questions related to the Michigan EMS Protocal Suite. How can I help you today?")
with st.chat_message("user"):
    st.write(user_question)
with st.chat_message("ai"):
    st.write(cortex_reponse)

## :blue[Step 4: Deploy an Interactive Chat Experiance for Users with Streamlit in Snowflake] :balloon:

With all the foundational pieces in place, we can now securely deploy our Chat Application directly in Snowflake with Streamlit.

We can deploy the Streamlit in Snowflake app directly from this notebook!


#### First, let's create a stage to hold our Streamlit Application files.
** Note: This will no longer be needed once Multi-page editing is available in Streamlit in Snowflake. At that time, all app files will be contained directly in the Streamlit Object for newly created apps.  
- [Work with files in Streamlit in Snowflake](https://docs.snowflake.com/en/LIMITEDACCESS/streamlit/work-with-files)

In [ ]:
-- variable for name of Snowflake stage
SET st_stage_name = '{{fully_qualified_solution_schema}}.'||'mi_ems_proto_app';

-- create stage if not exists
CREATE OR REPLACE STAGE IDENTIFIER($st_stage_name)
    DIRECTORY = (ENABLE = TRUE)
    COMMENT = 'Stage for Streamlit App files';

#### Now we will load the streamlit app files to the stage we just created.

In [ ]:
# Specify the path you want to start from and the directory name to match
document_dir = 'streamlit_app'
path = os.getcwd()
match_name = document_dir

# Call the function to get a list of matching directories
matching_directories = list_matching_directories_recursively(path, match_name)

# return the path to the first matching directory (assumption is there is only one)
st_files_path = matching_directories[0]
st.info(f"""The Notebook filesystem path to the streamlit files folder is:

**`{st_files_path}`**""")
st.info(f"Files available in the **`{document_dir}`** folder:")
st.write(os.listdir(st_files_path))


# Define the stage location 
st_stage = f"""@{fully_qualified_solution_schema}.streamlit"""

# Define the files you want to upload to a stage
st_file = "*" # '*' is a wildcard for all files in the directory 

# Define the path
st_stage_location = "/"+os.path.split(st_files_path)[-1]

# executing
print("Executing...")
print(f"""session.file.put({st_files_path}/{st_file}, {st_stage}, auto_compress=False, overwrite=True)""")

# Push the files to the stage with a PUT operation
put_result = session.file.put(f"{st_files_path}/{st_file}", 
                              f"{st_stage}", 
                                auto_compress=False, 
                                overwrite=True)

# Convert the result to a pandas DataFrame for readability
df = pd.DataFrame(put_result)

# Print the results
st.write("**Streamlit Files Uploaded:**")
st.write(df)

#### Finally, the code below will deploy the chat application directly in Snowflake.

In [ ]:
-- channge the team name!
SET team_name = 'SUPERSTARS';
SET app_name = $team_name || '_' || 'MI_EMS_PROTOCAL_ASSISTANT';
SET app_title = $team_name || '-' || 'Michigan EMS Protocol Assistant';

-- note, this is the older streamlit create syntax
CREATE OR REPLACE STREAMLIT IDENTIFIER($app_name)
    ROOT_LOCATION = {{st_stage}}
    MAIN_FILE = '/streamlit_main.py'
    QUERY_WAREHOUSE = {{query_warehouse}}
    TITLE = $app_title;